In [ ]:
import aduq.pyam2 as am2
import os 
import numpy as np

data_path = "aduq/pyam2/data/"

feed = am2.IO.load_dig_feed(os.path.join(data_path, "influent_state.csv"))[::24][:100]
ini_state = am2.IO.load_dig_state(os.path.join(data_path, 'init_state.json'))

param = am2.IO.load_dig_param(os.path.join(data_path, "parameter.json"))

AM2 manipulates np.ndarray objects, which can be seen as pandas object using the appropriate functions.

In [ ]:
print(f"Digester feed:\n{am2.IO.feed_to_pd(feed)}")

print(f"Digester State:\n{am2.IO.state_to_pd(ini_state)}")
print(f"Digester Parameter:\n{am2.IO.param_to_pd(param)}")

## Basic operations

AM2 model is called through run_am2

In [ ]:
pred = am2.run_am2(param=param, influent_state=feed, initial_state=ini_state)

The derivative can be computed using the am2_derivative function. It is much more stable than the adm1 counterpart, since it computes the derivative by differentiating under the integral the ODE. As such, the derivative can be computed efficiently at the same time as the output (function am2_with_der). The derivative with respect to all the parameters is computed, then translated to the derivative with respect to the required parameters. 

Note that the derivative of $Z$ with respect to the parameters is always 0 (as is theoretically the case). In practice, $Z$ might experience variations due to the ODE solver, of small ($\leq 10^{-6}) amplitudes.

In [ ]:
params_to_calib = ['mu1max', "KS1"]
am2_der = am2.am2_derivative(
    param=param,
    params_to_der=params_to_calib,
    influent_state=feed,
    initial_state=ini_state,
    log_am2=False
)

# One can assess the quality of the derivative in the following fashion
import numpy as np

param_mod = param.copy()
pred = am2.run_am2(param, feed, ini_state)
param_pd = am2.IO.param_to_pd(param_mod)
perturb = np.random.normal(0,10 ** (-5), 2)
param_pd[params_to_calib] += perturb # param_mod is modified by side effect

pred_perturb = am2.run_am2(param_mod, feed, ini_state)
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    ratios = np.array((pred_perturb - pred))[:, 1:] / np.tensordot(perturb, am2_der, (0, 0)) 

print(f"Maximum discrepancy should be small: {np.max(np.abs(ratios[:, [0, 1, 2, 3, 5, 6, 7]] - 1))}")
print(f"Column for Z outputs 0/0 ratio: {np.all(np.isnan(ratios[:, 4]))}")

Due to the small number of parameters, it is always assumed that all parameters are calibrated and no sensitivity analysis module is prepared.

## Optimisation routines

In [ ]:
obs = pred.copy()
obs[:, 1:] = obs[:, 1:] * np.exp( np.random.uniform(-0.1, 0.1, obs[:, 1:].shape) )

import matplotlib.pyplot as plt
obs_pd = am2.IO.states_to_pd(obs)
pred_pd = am2.IO.states_to_pd(pred)
plt.plot(pred_pd["time"], pred_pd["X1"])
plt.plot(obs_pd["time"], obs_pd["X1"])
plt.show()
print(f"Prediction error: {am2.am2_err(pred, obs)}")

In [ ]:
init_param = param.copy()
init_param = init_param  *  np.random.uniform(
    0.5, 2, len(init_param)
) 

out = am2.optim.optim_cma_am2(
    init_param=init_param,
    obs=obs,
    chain_length=24,
    influent_state=feed,
    initial_state=ini_state,
    per_step=8, 
    print_rec=8,
    radius_factor=0.8,
    radius_ini=.6
)
opti_param = out.opti_param

import matplotlib.pyplot as plt

plt.plot(out.hist_score)

In [ ]:
optim_res = am2.optim.am2_vi(
    obs=obs,
    influent_state=feed,
    initial_state=ini_state,
    temperature=0.01,
    chain_length=40,
    per_step=20,
    step_size=0.01,
    gen_decay=0.05,
    momentum=0.8,
    print_rec=4,
    parallel=True,
)

## UQ module

### Fisher's information

In [ ]:
fim_out = am2.UQ.am2_fim(opti_param, obs, feed, ini_state, silent=True)
am2.UQ.am2_fim_pval(param, opti_param, cov = fim_out["cov"], inv_cov = fim_out["fisher"] )

In [ ]:
# And one can evaluate the uncertainty on the previsions using linear transfer of uncertainty and gaussian hypothesis

output_UQ = am2.UQ.am2_fim_pred(
    opti_predict=fim_out["opti_predict"],
    cov=fim_out["cov"],
    der_log_am2=fim_out["der_log_am2"],
    conf_lev=0.7,
)

low_quant = am2.IO.states_to_pd(output_UQ["lower_quant"])
high_quant = am2.IO.states_to_pd(output_UQ["upper_quant"])
plt.plot(low_quant["time"], low_quant["S1"], label="lower prediction quantile")
plt.plot(high_quant["time"], high_quant["S1"], label="higher prediction quantile")
plt.legend()
plt.ylabel(f"S1 (in {am2.IO.predict_units_dict['S1']})")
plt.show()

In [ ]:
beale_out = am2.UQ.am2_beale(20, conf_lev=.99, cov=fim_out['cov'], param=opti_param, params_eval=params_to_calib, obs=obs,influent_state=feed, initial_state=ini_state)

In [ ]:
plt.plot(beale_out["boundary"]["mu1max"], beale_out["boundary"]["KS1"], ".")
plt.xlabel(f"mu1max (in {am2.IO.parameter_units['mu1max']})")
plt.ylabel(f"KS1 (in {am2.IO.parameter_units['KS1']})")
plt.show()

In [ ]:
from aduq.pyam2.UQ.bootstrap import am2_lin_bootstrap
lin_boot_out = am2_lin_bootstrap(10**4, obs=obs, opti_param=opti_param, params_eval=params_to_calib, influent_state=feed, initial_state=ini_state)

In [ ]:
import pandas as pd
sample_boot = lin_boot_out["sample"]
sample_pd = pd.DataFrame(sample_boot, columns = list(am2.IO.parameter_dict.keys()))
plt.plot(sample_pd["mu1max"], sample_pd["KS1"], '.')

In [ ]:
boot_out = am2.UQ.am2_bootstrap(
    20,
    opti_param =opti_param, obs=obs,
    influent_state=feed, initial_state=ini_state,
    chain_length=8, per_step=20, radius_ini=.1)

In [ ]:
sample_full_boot_pd = pd.DataFrame(np.array(boot_out), columns = list(am2.IO.parameter_dict.keys()))

In [ ]:
plt.plot(sample_full_boot_pd["mu1max"], sample_full_boot_pd["KS1"], '.')